In [18]:
import pandas as pd
from pathlib import Path
import csv
import glob
import os


In [14]:
# INPUT:
#--------

path = Path('/home/isischameleon/Dropbox/Coding/gitrepos/abm_commons_tragedy')

# multiple files at a time
list_original_csv_files = ['network-commons experiment #3 nb-villagers link-transmission network-type-table.csv']

#only one file at a tim
original_csv_file = 'network-commons experiment #3 nb-villagers link-transmission network-type-table.csv'

maximum_run_steps = 10000

In [15]:
# SAME AS NETWORK-COMMONS-ANALYSIS.IPYNB
#----------------------------------------
# global parameters related to the structure of the file
# other global variables (like 'path')

col_names_index = ['[run number]','[step]' ]
col_names_parameters = ['nb-villagers', 'LINK-TRANSMISSION-DISTANCE','min-degree', 'adaptive-harvest?',
                       'network-type', 'MIN-RSC-SAVING-PCT', 'wiring-probability',
                       'regrowth-chooser', 'INIT-HARVEST-LEVEL','DECREASE-PCT', 'INCREASE-PCT',
                       'MAX-TURTLE-BACKPACK','HFC-TICKER-MAX','HFC-TICKER-STOP','HFC-TICKER-START',
                        'PRL-TICKER-MAX','PRL-TICKER-STOP','PRL-TICKER-START','FACTOR-DIV',
                        'PERCENT-BEST-LAND','regrowth-chooser','MAX-TURTLE-VISION', 'turtle-die?'
                       ]
col_names_reporters = ['total-resource-reporter','total-patch-regrowth','total-turtle-resource-reporter',
                       'total-quantity-harvested','number-of-hungry-turtles','total-wealth','total-food-exchanged']

col_names_reporters_group = ['group-turtle-resource','group-turtle-wealth','group-turtle-prl',
                             'group-turtle-hfc']
col_names_useless = ['show-link?','debugging-agentset?', 'debugging-agentset-nb','color-chooser',
                     'DEBUG-RATE','DEBUG','TURTLE-PROC-CHOOSER']
experiment_name=""

In [101]:
# Functions : reading csv header and csv file run number by run number 
#---------------------------------------------------------

def read_netlogo_header(csv_name: str):
    
    filename = path/'data'/csv_name
    
    df = pd.read_csv(filename, nrows=6, sep=',',
                usecols=lambda x: x not in col_names_useless )
    
    return df

def read_netlogo_csv_one_run_at_a_time_generator(csv_name: str, max_run_steps: int):

    filename = path/'data'/csv_name

    # Find experiment name: read line 3 for experiment name (--> i== 2)

    with open(filename, "r") as f:
        reader = csv.reader(f, delimiter=",")
        for i, line in enumerate(reader):
            if i == 2:
                experiment_name = line[0]
                break
                
    fileEmpty = False
    skiprows = 6
                
    while (fileEmpty == False):           

        # read whole file into a pandas discarding useless columns
        
        print("reading {0} rows in filename {1} from row {2}".format(max_run_steps, filename, skiprows))

        df = pd.read_csv(filename, skiprows=skiprows, nrows=10000, sep=',',
                    usecols=lambda x: x not in col_names_useless )
        
        if (len(df) < 10000): fileEmpty = True


        df.insert(0, 'experiment_name', experiment_name)
        
        print("df before setting the index:", df[['PCT-INCREASE']].head())
        
        df.set_index(['experiment_name']+col_names_index, append=False, inplace=True)
        df_run.sort_index(inplace=True)

        # access the value of index line 0 index 2 which is the run nunber  (0 = experiment, 1 = run, 2 = step)
        run_number = df.index[0][1]
        print("Processing experiment {0} run number {1}".format(experiment_name, run_number))

        returning_df = df_run.query('`experiment_name` == @experiment_name and `[run number]` == @run_number')
        
        skiprows = skiprows + len(returning_df) - 1
        
        df.reset_index(drop=False, inplace=True)
        df = df[0:0]
        
        print("clean up df:", df.head())
    
        yield returning_df

In [102]:
#  MAIN 
#------
# File being read : original_csv_file
# 1) Reading file header

df_header = read_netlogo_header(original_csv_file)

df_header.head()

i = 0
for df in read_netlogo_csv_one_run_at_a_time_generator(original_csv_file, 10000):
    #global_df = pd.concat([df_header, df], axis=0)
    df.head()
    if (i == 5): break
    i = i+1

reading 10000 rows in filename /home/isischameleon/Dropbox/Coding/gitrepos/abm_commons_tragedy/data/network-commons experiment #3 nb-villagers link-transmission network-type-table.csv from row 6


KeyError: "None of [Index(['PCT-INCREASE'], dtype='object')] are in the [columns]"

In [49]:
df_run = next(read_netlogo_csv_one_run_at_a_time_generator(original_csv_file, 10000))

Processing experiment experiment #3 nb-villagers link-transmission network-type run number 1


In [50]:
df_run.head()

DECREASE-PCT  \
experiment_name                                    [run number] [step]                 
experiment #3 nb-villagers link-transmission ne... 1            0                  1   
                                                                1                  1   
                                                                2                  1   
                                                                3                  1   
                                                   2            0                  1   

                                                                        min-degree  \
experiment_name                                    [run number] [step]               
experiment #3 nb-villagers link-transmission ne... 1            0                2   
                                                                1                2   
                                                                2                2   
                                                                3                2   
                                                   2            0                2   

                                                                        adaptive-harvest?  \
experiment_name                                    [run number] [step]                      
experiment #3 nb-villagers link-transmission ne... 1            0                    True   
                                                                1                    True   
                                                                2                    True   
                                                                3                    True   
                                                   2            0                    True   

                                                                        MAX-TURTLE-BACKPACK  \
experiment_name                                    [run number] [step]                        
experiment #3 nb-villagers link-transmission ne... 1            0                         4   
                                                                1                         4   
                                                                2                         4   
                                                                3                         4   
                                                   2            0                         4   

                                                                        INIT-HARVEST-LEVEL  \
experiment_name                                    [run number] [step]                       
experiment #3 nb-villagers link-transmission ne... 1            0                        1   
                                                                1                        1   
                                                                2                        1   
                                                                3                        1   
                                                   2            0                        1   

                                                                        INCREASE-PCT  \
experiment_name                                    [run number] [step]                 
experiment #3 nb-villagers link-transmission ne... 1            0                0.1   
                                                                1                0.1   
                                                                2                0.1   
                                                                3                0.1   
                                                   2            0                0.1   

                                                                        HFC-TICKER-MAX  \
experiment_name                                    [run number] [step]                   
experiment #3 nb-villagers link-transmission ne... 1            0             

In [86]:
#
for experiment_name, new_df in df_run.groupby(level=0):
    print(new_df[['INCREASE-PCT']])

AttributeError: 'DataFrameGroupBy' object has no attribute 'groupby'

In [66]:
# Test : find all the index values at a specific level,e.g. find all the run numbers values at level 1
#https://stackoverflow.com/questions/18835077/selecting-from-multi-index-pandas

df_run.sort_index(inplace=True)
index = df_run.index
print(index.names)
print(index.get_level_values(1))
print(index.to_numpy)

df_run2 = df_run.iloc[df_run.index.get_level_values(1) == 1]

['experiment_name', '[run number]', '[step]']
Int64Index([1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
            ...
            5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
           dtype='int64', name='[run number]', length=10000)
<bound method IndexOpsMixin.to_numpy of MultiIndex([('experiment #3 nb-villagers link-transmission network-type', ...),
            ('experiment #3 nb-villagers link-transmission network-type', ...),
            ('experiment #3 nb-villagers link-transmission network-type', ...),
            ('experiment #3 nb-villagers link-transmission network-type', ...),
            ('experiment #3 nb-villagers link-transmission network-type', ...),
            ('experiment #3 nb-villagers link-transmission network-type', ...),
            ('experiment #3 nb-villagers link-transmission network-type', ...),
            ('experiment #3 nb-villagers link-transmission network-type', ...),
            ('experiment #3 nb-villagers link-transmission network-type', ...),
            ('experiment #3 nb-v

In [78]:
# tests 
#------
df_run2.tail()
df_run_line_436 = df_run.iloc[df_run.index.get_level_values(2) == 1000]
df_run_line_436.head()

In [80]:
#Tests
#------
#https://stackoverflow.com/questions/18835077/selecting-from-multi-index-pandas

df_run.query('`[run number]` >= 1 and `[step]` >= 50')

experiment_name = 'experiment #3 nb-villagers link-transmission network-type'
run_number = 1

df_run.query('`experiment_name` == @experiment_name and `[run number]` == @run_number')

DECREASE-PCT  \
experiment_name                                    [run number] [step]                 
experiment #3 nb-villagers link-transmission ne... 1            0                  1   
                                                                1                  1   
                                                                2                  1   
                                                                3                  1   
                                                                4                  1   
...                                                                              ...   
                                                                432                1   
                                                                433                1   
                                                                434                1   
                                                                435                1   
                                                                436                1   

                                                                        min-degree  \
experiment_name                                    [run number] [step]               
experiment #3 nb-villagers link-transmission ne... 1            0                2   
                                                                1                2   
                                                                2                2   
                                                                3                2   
                                                                4                2   
...                                                                            ...   
                                                                432              2   
                                                                433              2   
                                                                434              2   
                                                                435              2   
                                                                436              2   

                                                                        adaptive-harvest?  \
experiment_name                                    [run number] [step]                      
experiment #3 nb-villagers link-transmission ne... 1            0                    True   
                                                                1                    True   
                                                                2                    True   
                                                                3                    True   
                                                                4                    True   
...                                                                                   ...   
                                                                432                  True   
                                                                433                  True   
                                                                434                  True   
                                                                435                  True   
                                                                436                  True   

                                                                        MAX-TURTLE-BACKPACK  \
experiment_name                                    [run number] [step]                        
experiment #3 nb-villagers link-transmission ne... 1            0                         4   
                                                                1                         4   
                                                                2                         4   
                                                                3                         4   
                            